# Building a Simple Customer Support Agent

This notebook demonstrates how to build an autonomous agent using an LLM and Tool Use (Function Calling). The agent can interact with a database to help customers with orders, refunds, and tracking.

## Core Capabilities

* **Business Logic Integration:** Directly connects to database helpers for customer data.
* **Intelligent Tool Selection:** The agent analyzes user intent to choose the right function.
* **Structured Data Processing:** Handles JSON inputs/outputs for reliable data flow.
* **Natural Language Generation:** Converts technical database results into friendly, human-style responses.

In [ ]:
%pip install openai python-dotenv

In [ ]:
# Setup with z.ai and .env
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import sqlite3
from datetime import datetime
from typing import Dict, Any

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client (points to z.ai)
client = OpenAI(
    base_url=os.environ.get("ZAI_BASE_URL"),
    api_key=os.environ.get("ZAI_API_KEY")
)

MODEL = os.environ.get("ZAI_MODEL", "gpt-4o")

print("✅ Using LLM via z.ai")
print(f"🔗 Endpoint: {os.environ.get('ZAI_BASE_URL')}")
print(f"🤖 Model: {MODEL}")
print()

In [ ]:
# Initialize Database
import importlib
import db_helper
importlib.reload(db_helper)
from db_helper import reset_database

reset_database()

In [ ]:
# Agent Definition
from db_helper import (
    process_refund, 
    get_customer_orders, 
    check_order_status, 
    lookup_customer
)
from db_tools import tools

def customer_support_agent(user_message: str):
    """
    The main Agent Loop. It follows this flow:
    1. Receive user input.
    2. Call the LLM with available tools.
    3. If the LLM wants to use a tool, execute the Python function.
    4. Pass the result back to the LLM.
    5. Repeat until the LLM provides a final text answer.
    """
    
    system_prompt = """You are a helpful customer support agent. You can:
                        - Look up customer information by customer ID
                        - Check order status for specific orders
                        - View all orders for a customer
                        - Process refunds for delivered orders
                        
                        Be polite, professional, and helpful. Always verify information before taking actions.
                        If you need to look up a customer's orders, ask for their customer ID first. 
                        IMPORTANT: Before processing any refund, you MUST first check the order status 
                        to verify the order exists and is in "Delivered" state. Only delivered orders can be refunded."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    print(f"👤 Customer: {user_message}\n")
    
    # Initial call to the model
    response = client.chat.completions.create(
        model=MODEL,
        max_tokens=1024,
        messages=messages,
        tools=tools
    )

    print("="*80)
    print(f"🐛 Model Response: {[{'finish_reason': c.finish_reason, 'reasoning': c.message.reasoning_content} for c in response.choices]}\n")
    print("="*80)
    
    # The "Agent Loop": Continue as long as the model wants to use tools
    while response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        
        # Record the assistant's request to use a tool
        messages.append(assistant_message)
        
        # Process each tool use request in the response
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_input = json.loads(tool_call.function.arguments)
            
            print(f"🔍 Agent decided to use: {tool_name}")
            print(f"   Args: {tool_input}")
            
            # Routing logic: Map tool names to actual Python functions
            if tool_name == "lookup_customer":
                result = lookup_customer(**tool_input)
            elif tool_name == "check_order_status":
                result = check_order_status(**tool_input)
            elif tool_name == "get_customer_orders":
                result = get_customer_orders(**tool_input)
            elif tool_name == "process_refund":
                result = process_refund(**tool_input)
            else:
                result = {"error": "Unknown function"}
            
            print(f"📊 Tool Result: {json.dumps(result, indent=2)}\n")
            
            # Add the tool result to the conversation
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })
        
        # Call the model again with the new context
        response = client.chat.completions.create(
            model=MODEL,
            max_tokens=1024,
            messages=messages,
            tools=tools
        )
    
    # Final step: Extract and display the text response
    final_message = response.choices[0].message.content
    
    print(f"🤖 Agent: {final_message}")
    return final_message

## 🧪 Testing Scenarios

### 1. Basic Information Retrieval

Testing if the agent can identify a single order status.

In [ ]:
# Test - Simple order lookup
customer_support_agent("Hi, can you check the status of order ORD12345?");

### 2. Single Tool Call: Customer Orders Lookup
Testing if the agent can retrieve all orders for a given customer ID using a single tool call (`get_customer_orders`).

In [ ]:
# Test - Customer with all orders
customer_support_agent("My customer ID is CUST003. Can you show me all my orders?");

### 3. Two-Step Task: Refund Request
Testing if the agent can process a refund request. The agent may first check the order status, then process the refund if the order is in "Delivered" state. This requires 1-2 tool calls.

In [ ]:
# Test - Refund request
reset_database()

customer_support_agent("I want to return order ORD12347. The product didn't match the description.");


### 4. Multi-Step Task: Orders + Conditional Refunds
This is the most advanced test. The agent must:

1. Retrieve all orders for the customer (`get_customer_orders`)
2. Analyze the results to find orders with "Delivered" status
3. Process a refund for each delivered order (`process_refund` - called multiple times)

This requires multiple tool calls and reasoning between each step.

In [ ]:
# Test - Complex multi-step
reset_database()

customer_support_agent("Hi, I'm customer CUST001. Show me my orders and process a refund for any delivered items.");